## Load

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
tf.debugging.set_log_device_placement(False)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


## Dataset

In [4]:
cifar = keras.datasets.cifar10

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [6]:
x_train.shape

(50000, 32, 32, 3)

In [7]:
x_train, x_test = x_train / 255.0, x_test / 255.0

## Sequential

In [8]:
model = keras.Sequential(
    [
      keras.Input(shape = (32,32,3)),
      layers.Conv2D(32, 3, padding='valid', activation = 'relu'), # (how_many_outs, kernel_size, padding, activation),
      layers.MaxPooling2D(pool_size = (2,2)),
      layers.Conv2D(64, 3, activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(128, 3, activation='relu'),
      layers.Flatten(),
      
      layers.Dense(64, activation = 'relu'),
      layers.Dense(10),
     
    ]
)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0

In [10]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits="True"),
    optimizer = keras.optimizers.Adam(lr=1e-3),
    metrics = ["accuracy"]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(
    x_train, 
    y_train, 
    batch_size = 64, 
    epochs = 100, 
    validation_split=0.1, 
    validation_batch_size=64, 
    verbose = 2
)

In [ ]:
model.evaluate(x_test, y_test, batch_size = 64, verbose = 2)

157/157 - 1s - loss: 3.7034 - accuracy: 0.6847 - 748ms/epoch - 5ms/step


[3.7033956050872803, 0.6847000122070312]

## Functional

In [45]:
def my_model():
  inputs = keras.Input(shape = (32,32,3))
  x = layers.Conv2D(32,3, padding='same')(inputs)
  x = layers.Dropout(0.3)(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPooling2D()(x)
  x = layers.Conv2D(64,5, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Conv2D(128,3)(x)
  x = layers.Dropout(0.1)(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)

  x = layers.Flatten()(x)
  x = layers.Dense(64, activation = 'relu')(x)
  outputs = layers.Dense(10)(x)

  model = keras.Model(inputs = inputs, outputs = outputs)
  return model


In [46]:
model = my_model()

In [47]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits="True"),
    optimizer = keras.optimizers.Adam(lr=1e-3),
    metrics = ["accuracy"]
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [48]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_29 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 dropout_3 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_25 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 tf.nn.relu_25 (TFOpLambda)  (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                      

In [ ]:
model.fit(
    x_train, 
    y_train, 
    batch_size = 64, 
    epochs = 100, 
    validation_split=0.1, 
    validation_batch_size=64, 
    verbose = 2
)

Epoch 1/100
704/704 - 5s - loss: 2.3292 - accuracy: 0.1025 - val_loss: 2.3027 - val_accuracy: 0.0970 - 5s/epoch - 7ms/step
Epoch 2/100
704/704 - 4s - loss: 2.3027 - accuracy: 0.0999 - val_loss: 2.3028 - val_accuracy: 0.0950 - 4s/epoch - 6ms/step
Epoch 3/100


In [44]:
model.evaluate(x_test, y_test, batch_size = 64, verbose = 2)

157/157 - 0s - loss: 1.6643 - accuracy: 0.6334 - 432ms/epoch - 3ms/step


[1.6643060445785522, 0.633400022983551]